In [1]:
# Step 1

In [2]:
# prompt: write code for google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ⚡ Quick Setup - Run after runtime reset (CPU/GPU Switch)
# Installs essential packages silently to save output clutter

!pip install dask pytz torch pytorch-forecasting pytorch-lightning \
    rich colorama matplotlib seaborn pandas numpy tensorboard \
    'lightning[extra]' pyarrow fastparquet --quiet

print("\033[92m✅ All required packages installed successfully.\033[0m")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 363.4/363.4 MB 113.8 MB/s eta 0:00:01

In [ ]:
import pytorch_forecasting
print(pytorch_forecasting.__version__)

In [ ]:
# Standard Library
import os
import datetime
import glob
import json
import shutil
import math
import pytz
from concurrent.futures import ThreadPoolExecutor

# Third-Party Libraries
import numpy as np
import pandas as pd
import dask.dataframe as dd
import matplotlib.pyplot as plt
import seaborn as sns

import torch

# PyTorch Lightning
# from datetime import datetime
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks import ModelCheckpoint

# PyTorch Forecasting
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import RMSE

ist = pytz.timezone('Asia/Kolkata')
now_ist = datetime.datetime.now(ist)
timestamp = now_ist.strftime("%Y%m%d-%H%M%S")
print(f"All Libraries are loaded : {timestamp}")

In [ ]:
# --------- Main Processing ---------

vm_folders = sorted([
    name.split('=')[1] for name in os.listdir(parquet_path) if name.startswith("VM=")
])

print(f"Available VMs: {vm_folders[:10]} ... Total: {len(vm_folders)}")

# Load First N VMs Dynamically [100, 250, 500, 750, 1000, 1250]

for N in [num_vms_to_load]:
    selected_vms = vm_folders[:N]

    df3 = dd.read_parquet(
        parquet_path,
        filters=[("VM", "in", selected_vms)]
    ).compute()

    print(f"✅ Loaded {N} VMs → Shape: {df3.shape}")

# print(df3.columns.tolist())
# print(df3[['Timestamp', 'time_idx']].tail())

# --- Column Change ---
df3 = df3.rename(columns={'VM': 'vm_id'})

tft_df = df3.dropna(subset=[
    'cpu_utilization_ratio',
    'memory_utilization_ratio',
    'disk_total_throughput',
    'network_total_throughput'
])

# Convert Dask to Pandas
tft_df = tft_df.compute() if 'dask' in str(type(tft_df)) else tft_df

print(f"📊 Columns available before filtering: {len(tft_df.columns)}")
print(tft_df.columns.tolist())


In [ ]:
# ------------------ Dynamic Config with Dataset Size Labels ------------------

import multiprocessing

# Automatically detect how many CPU cores are available
num_cpu_cores = multiprocessing.cpu_count()

# Heuristic rule: use half the cores, but keep between 2 and 8
dynamic_num_workers = min(max(num_cpu_cores // 2, 2), 8)

# Update in training config
train_config["num_workers"] = dynamic_num_workers

print(f"✅ Using {dynamic_num_workers} workers for DataLoader (detected {num_cpu_cores} cores)")

# Calculate dataset scale
current_vm_rows = len(tft_df)
vm_group = tft_df.groupby("vm_id")
rows_per_vm = vm_group.size().mean()

# Label the dataset size
if current_vm_rows < 200_000:
    data_scale_label = "SMALL"
    lr = 0.01
    hidden = 8
    dropout = 0.3
elif current_vm_rows < 1_000_000:
    data_scale_label = "MEDIUM"
    lr = 0.005
    hidden = 16
    dropout = 0.2
elif current_vm_rows < 5_000_000:
    data_scale_label = "LARGE"
    lr = 0.003
    hidden = 32
    dropout = 0.1
else:
    data_scale_label = "XLARGE"
    lr = 0.002
    hidden = 64
    dropout = 0.1

# Encoder length
def get_encoder_length(row_count):
    if row_count >= 5000:
        return 400
    elif row_count >= 3000:
        return 300
    elif row_count >= 2000:
        return 200
    elif row_count >= 1000:
        return 100
    elif row_count >= 600:
        return 60
    else:
        return 30

dynamic_encoder_length = get_encoder_length(rows_per_vm)

# Batch size heuristic
estimated_batch_size = int(min(max(0.001 * current_vm_rows, 16), 128))

# Print final info
print(f"\n📦 Dataset Scale: {data_scale_label}")
print(f"📊 Total rows: {current_vm_rows}, Avg rows/VM: {rows_per_vm:.0f}")
print(f"✅ Encoder Length: {dynamic_encoder_length}")
print(f"✅ Config -> Batch: {estimated_batch_size}, LR: {lr}, Hidden: {hidden}, Dropout: {dropout}")

In [ ]:
# --------- User Configurable Parameters ---------

# Data path & VM selection
parquet_path = "/content/drive/MyDrive/datasets/processed/FeatureEngcolab"

# e.g., 50, 100, 250 etc.
num_vms_to_load = 30

# Model training parameters
train_config = {
    "targets": ['cpu_utilization_ratio'],
    "time_varying_known_reals": ['time_idx', 'hour_sin', 'hour_cos', 'dayofweek_sin', 'dayofweek_cos', 'month_sin', 'month_cos'],
    "time_varying_unknown_reals": ['CPU usage [%]','Memory usage [KB]',
    'Disk read throughput [KB/s]', 'Disk write throughput [KB/s]','Network received throughput [KB/s]', 'Network transmitted throughput [KB/s]',
    'cpu_util_percent', 'memory_util_percent','disk_rolling_mean', 'disk_rolling_std','network_rolling_mean', 'network_rolling_std'],
    "group_ids": ['vm_id'],
    "max_encoder_length": dynamic_encoder_length,
    "max_preditction_length": 2016,
    "hidden_size": hidden,
    "dropout": dropout,
    "learning_rate": lr,
    "batch_size": estimated_batch_size,
    "num_workers": dynamic_num_workers,
    "early_stopping_patience": 3,
    "epochs": 1,
    "loss_fn": RMSE(),
    "output_base_dir": "/content/drive/MyDrive/output",
    "log_dir": "/content/drive/MyDrive/output/logs"
}

# VM count for folder naming (update if needed)
vm_count = f"{num_vms_to_load}VMs"

In [ ]:
# --------- Helper: Generate Run Folder Name ---------

def get_run_folder_name(train_config, vm_count):
    target = "cpu"
    past = f"past{train_config['max_encoder_length']}"
    fut = f"fut{train_config['max_prediction_length']}"
    batch = f"bs{train_config['batch_size']}"
    lr = f"lr{train_config['learning_rate']:.0e}".replace('+0', '')
    hid = f"hid{train_config['hidden_size']}"

    return f"{target}_{vm_count}_{past}_{fut}_{batch}_{lr}_{hid}_{timestamp}"

In [ ]:
# ✅ Drop unused columns - place this here
columns_to_keep = (
    train_config["time_varying_known_reals"]
    + train_config["time_varying_unknown_reals"]
    + train_config["targets"]
    + train_config["group_ids"]
    + ['time_idx', 'timestamp']
)
columns_to_keep = list(set(columns_to_keep))  # Remove duplicates if any
tft_df = tft_df[columns_to_keep]

print(f"✅ Columns after filtering: {len(tft_df.columns)}")

In [ ]:
# Setup output and log folders based on run config

folder_name = get_run_folder_name(train_config, vm_count)

train_config["output_base_dir"] = os.path.join(train_config["output_base_dir"], folder_name)
train_config["log_dir"] = os.path.join(train_config["log_dir"], folder_name)

os.makedirs(train_config["output_base_dir"], exist_ok=True)
os.makedirs(train_config["log_dir"], exist_ok=True)

print("Output directory:", train_config["output_base_dir"])
print("Log directory:", train_config["log_dir"])

# Now you can proceed to model training using `tft_df` and `train_config` as usual.

In [ ]:
import math

# Basic Dataset Summary
print(f"\n✅ Dataset Summary:")
print(f"- Min time_idx      : {tft_df['time_idx'].min()}")
print(f"- Max time_idx      : {tft_df['time_idx'].max()}")
print(f"- Total rows        : {len(tft_df):,}")
print(f"- Batch size config : {train_config['batch_size']}")

# Train/Val Split Calculation
max_time_idx = tft_df['time_idx'].max()
split_point = max_time_idx * 0.8

train_rows = (tft_df['time_idx'] <= split_point).sum()
val_rows = (tft_df['time_idx'] > split_point).sum()
total_rows = len(tft_df)
val_pct = 100 * val_rows / total_rows

print(f"\n✅ Train/Validation Split:")
print(f"- Split point (time_idx > {split_point:.2f})")
print(f"- Train rows       : {train_rows:,}")
print(f"- Validation rows  : {val_rows:,} ({val_pct:.2f}%)")

# Important for model creation
val_df = tft_df[tft_df['time_idx'] > split_point]

# Validation Window Stats
val_points = len(val_df)
min_required = train_config['max_encoder_length'] + train_config['max_prediction_length']
total_val_windows = val_points - min_required + 1
expected_val_batches = math.ceil(total_val_windows / train_config['batch_size'])

print(f"\n✅ Validation Window Stats:")
print(f"- Validation points : {val_points:,}")
print(f"- Min required points: {min_required}")
print(f"- Total windows      : {total_val_windows:,}")
print(f"- Expected batches   : {expected_val_batches:,}")

In [ ]:
# Reset index (important for unique indexing)
tft_df = tft_df.reset_index(drop=True)

# Prepare TimeSeriesDataSet for training portion (80%)
dataset = TimeSeriesDataSet(
    tft_df[tft_df.time_idx <= tft_df['time_idx'].max() * 0.8],
    time_idx='time_idx',
    target=train_config["targets"][0],  # 'cpu_utilization_ratio' here
    group_ids=train_config["group_ids"],
    max_encoder_length=train_config["max_encoder_length"],
    max_prediction_length=train_config["max_prediction_length"],
    time_varying_known_reals=train_config["time_varying_known_reals"],
    time_varying_unknown_reals=train_config["time_varying_unknown_reals"]
    target_normalizer=GroupNormalizer(groups=train_config["group_ids"]),
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=True
)

# Validation dataset for prediction (no randomization, full data)
# ✅ "TimeSeriesDataSet applies sliding window logic on the training data,
# using the full configuration like past steps, future steps, groups, and prepares the dataset accordingly."

val_df = val_df.reset_index(drop=True) # Reset index of val_df

val_dataset = TimeSeriesDataSet.from_dataset(
    dataset, val_df, predict=True, stop_randomization=True
)

# Create dataloaders
# performs Batching, Padding, Time-aware slicing for forecasting

train_dataloader = dataset.to_dataloader(
    train=True,
    batch_size=train_config["batch_size"],
    num_workers=train_config["num_workers"]
)

val_dataloader = val_dataset.to_dataloader(
    train=False,
    batch_size=train_config["batch_size"],
    num_workers=train_config["num_workers"]
)


print(f"✅ Dataset and dataloaders ready. Train batches: {len(train_dataloader)}, Val batches: {len(val_dataloader)}")

In [ ]:
# Step 7

In [ ]:
# Logging & Callbacks
from pytorch_lightning.callbacks import EarlyStopping

for target in train_config["targets"]:
    print(f"\n🔁 Training for target: {target}")

    run_dir = os.path.join(train_config["output_base_dir"], f"{target}_run_{timestamp}")
    os.makedirs(run_dir, exist_ok=True)

    # Save Raw cleaned DF for inspection, tft_df is pandas dataframe.
    tft_df.to_csv(f"{run_dir}/tft_df.csv", index=False)

    # Save structured TimeSeriesDataset for future reuse
    dataset.save(f"{run_dir}/tft_df")

    # ✅ Save timestamp mapping if columns exist
    timestamp_cols_needed = ['vm_id', 'timestamp', 'time_idx']
    missing_cols = [col for col in timestamp_cols_needed if col not in val_df.columns]

    if missing_cols:
        print(f"⚠️ Skipping timestamp mapping export. Columns missing: {missing_cols}")
    else:
        timestamp_mapping_df = val_df[timestamp_cols_needed].reset_index(drop=True)
        timestamp_mapping_df.to_csv(f"{run_dir}/forecast_timestamp_mapping.csv", index=False)
        print(f"✅ Saved timestamp mapping to: {run_dir}/forecast_timestamp_mapping.csv")

    # Setup logging & checkpointing
    logger = CSVLogger(save_dir=train_config["log_dir"], name=f"{target}_log")

    checkpoint_callback = ModelCheckpoint(
        monitor="val_loss",
        dirpath=run_dir,
        filename="tft-{epoch:02d}-{val_loss:.2f}",
        save_top_k=1,
        save_last=True,
        mode="min"
    )

    early_stopping = EarlyStopping(
        monitor="val_loss",
        patience=train_config["early_stopping_patience"],
        mode="min"
    )

In [ ]:
# Step 8

In [ ]:
# Model, Lightning, Trainer

In [ ]:
import pytorch_lightning as pl

class TFTLightningModule(pl.LightningModule):
    def __init__(self, tft_model: TemporalFusionTransformer, learning_rate: float, loss_fn: torch.nn.Module):
        super().__init__()
        self.tft_model = tft_model
        self.learning_rate = learning_rate
        self.loss_fn = loss_fn

    def forward(self, x):
        return self.tft_model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat.prediction, y) # Extract prediction from output
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat.prediction, y) # Extract prediction from output
        self.log("val_loss", loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

# Create the TFT model
tft_model = TemporalFusionTransformer.from_dataset(
    dataset,
    learning_rate=train_config["learning_rate"],
    hidden_size=train_config["hidden_size"],
    dropout=train_config["dropout"],
    loss=train_config["loss_fn"],
    log_interval=10,
    reduce_on_plateau_patience=4
)

# Wrap the TFT model in a LightningModule
model = TFTLightningModule(
    tft_model=tft_model,
    learning_rate=train_config["learning_rate"],
    loss_fn=train_config["loss_fn"]
)

# Setup Trainer
if torch.cuda.is_available():
    accelerator = "gpu"
    devices = 1
else:
    accelerator = "cpu"
    devices = 1

trainer = Trainer(
    max_epochs=train_config["epochs"],
    accelerator=accelerator,
    devices=devices,
    logger=logger,
    callbacks=[checkpoint_callback, early_stopping],
    enable_checkpointing=True
)

# Fit the model
trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

In [ ]:
# Step 9

In [ ]:
# 🔍 Make raw predictions on validation set

In [ ]:
# Logging & Callbacks
from pytorch_lightning.callbacks import EarlyStopping

for target in train_config["targets"]:
    print(f"\n🔁 Training for target: {target}")

    run_dir = os.path.join(train_config["output_base_dir"], f"{target}_run_{timestamp}")
    os.makedirs(run_dir, exist_ok=True)

    # Save Raw cleaned DF for inspection (TFT input format)
    tft_df.to_csv(f"{run_dir}/tft_df.csv", index=False)

    # Save structured TimeSeriesDataset (structure, scalers, etc.)
    dataset.save(f"{run_dir}/tft_df_metadata")  # <-- .save stores dataset metadata

    if all(col in val_df.columns for col in meta_cols):
        meta_df = val_df[meta_cols].reset_index(drop=True)
        meta_df.to_csv(f"{run_dir}/forecast_metadata.csv", index=False)
        print(f"✅ Metadata saved to: {run_dir}/forecast_metadata.csv")
    else:
        print(f"⚠️ Skipping metadata save — columns not found: {meta_cols}")

    # Setup logging & checkpointing
    logger = CSVLogger(save_dir=train_config["log_dir"], name=f"{target}_log")

    checkpoint_callback = ModelCheckpoint(
        monitor="val_loss",
        dirpath=run_dir,
        filename="tft-{epoch:02d}-{val_loss:.2f}",
        save_top_k=1,
        save_last=True,
        mode="min"
    )

    early_stopping = EarlyStopping(
        monitor="val_loss",
        patience=train_config["early_stopping_patience"],
        mode="min"
    )

In [ ]:
# Predicted CPU Utilization next future steps based on above config.

In [ ]:
print(forecast)
print(type(forecast))
print(forecast.shape)

In [ ]:
# Step 10

In [ ]:
# Spike Detection & Save Metadata

In [ ]:
# 🔍 Simple spike detection based on 95th percentile threshold
spikes = forecast > np.percentile(forecast, 95)

# Save run notes and spike count
with open(f"{run_dir}/notes.txt", "w") as f:
    f.write(f"Target: {target}\n")
    f.write(f"Spikes > 95th percentile: {int(spikes.sum())}\n")
    f.write("Review plot.png and predictions.csv for further insights.\n")

# 💾 Save training config for reproducibility
# Create a serializable version of train_config
serializable_train_config = train_config.copy()
# Replace the non-serializable loss_fn object with its name
serializable_train_config["loss_fn"] = serializable_train_config["loss_fn"].__class__.__name__

with open(f"{run_dir}/modelconfig.json", "w") as f:
    json.dump(serializable_train_config, f, indent=2)

print(f"✅ Run complete. Outputs saved at: {run_dir}")

In [ ]:
forecast_path = os.path.join(run_dir, "predictions.csv")
print(f"✅ Forecast values saved to: {forecast_path}")

In [ ]:
# # Command to inspect .ckpt

# ckpt = torch.load("path/to/tft-epoch=01-val_loss=0.04.ckpt", map_location=torch.device('cpu'))
# print(ckpt.keys())

In [ ]:
# print(ckpt["epoch"])             # Epoch number
# print(ckpt["global_step"])       # Total steps
# print(ckpt["hyper_parameters"])  # Saved hyperparameters